# 4.8 Grouping Data & Aggregating Variables 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Creating path
path = r'C:\Users\avadr\27-01-2020 Instacart Basket Analysis'

In [3]:
# Importing data setfrom pickle format
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_derived.pkl'))

In [4]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy,most_orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy,average_orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,average_orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,both,Mid-range product,Regularly busy,slowest_days,most_orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,slowest_days,average_orders


## Step 2

In [5]:
# Using group by and agg to calculate mean for number of orders
ords_prods_merge.groupby('department_id').agg({'number_of_orders': ['mean']})

,number_of_orders
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Step 3

In my case, after doing the calculation on both subsets and complete data sets, the department number remained the same 21. The only difference I could notice was that it changed the mean for some of the departments; some went up, some went down, and a few remained the same.

## Step 4

In [6]:
# Creating new collumn containing max order number per customer
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['number_of_orders'].transform(np.max)

C:\Users\avadr\AppData\Local\Temp\ipykernel_15392\2581994623.py:2: FutureWarning: The provided callable <function amax at 0x000001CF4A651BC0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['number_of_orders'].transform(np.max)


In [7]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy,most_orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy,average_orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,average_orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,both,Mid-range product,Regularly busy,slowest_days,most_orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,slowest_days,average_orders,3


In [8]:
# Creating new columns containing loyalty_flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## Step 5

In [10]:
# Using group by and agg to calculate min, mean and max for number of orders to idetify spending diference in types of custmers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['min', 'mean', 'max']})

prices                    
                    min       mean      max
loyalty_flag                               
Loyal customer      1.0  10.386336  99999.0
New customer        1.0  13.294670  99999.0
Regular customer    1.0  12.495717  99999.0

 On average, new consumers spend the most money, followed by regular and then by loyal customers.

## Step 6

In [11]:
# Creating new collumn containing mean price per customer
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\avadr\AppData\Local\Temp\ipykernel_15392\116031512.py:2: FutureWarning: The provided callable <function mean at 0x000001CF4A6528E0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [12]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,...,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,both,Mid-range product,Regularly busy,Regularly busy,most_orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,both,Mid-range product,Regularly busy,Regularly busy,average_orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,both,Mid-range product,Busiest day,Busiest days,average_orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,both,Mid-range product,Regularly busy,slowest_days,most_orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,both,Mid-range product,Least busy,slowest_days,average_orders,3,New customer,4.972414


In [13]:
# Creating new columns containing spending_flag
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'low_spender'
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'high_spender'

In [14]:
ords_prods_merge[['user_id', 'mean_price', 'spending_flag']].head(20)

,user_id,mean_price,spending_flag
0,138,6.935811,low_spender
1,138,6.935811,low_spender
2,709,7.930208,low_spender
3,764,4.972414,low_spender
4,764,4.972414,low_spender
5,777,6.935398,low_spender
6,825,5.957576,low_spender
7,910,6.680000,low_spender
8,1052,7.162500,low_spender
9,1052,7.162500,low_spender


## Step 7

In [15]:
# Creating new collumn containing median days since prior order
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\avadr\AppData\Local\Temp\ipykernel_15392\3953925166.py:2: FutureWarning: The provided callable <function median at 0x000001CF4A797880> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [16]:
ords_prods_merge[['user_id', 'median_days_since_prior_order']].head(20)

,user_id,median_days_since_prior_order
0,138,8.0
1,138,8.0
2,709,8.0
3,764,9.0
4,764,9.0
5,777,11.0
6,825,20.0
7,910,6.0
8,1052,10.0
9,1052,10.0


In [17]:
# Creating new columns containing frequency_flag
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] <= 20) & (ords_prods_merge['median_days_since_prior_order'] > 10), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [18]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

In [19]:
ords_prods_merge[['user_id', 'median_days_since_prior_order', 'frequency_flag']].head(20)

,user_id,median_days_since_prior_order,frequency_flag
0,138,8.0,Frequent customer
1,138,8.0,Frequent customer
2,709,8.0,Frequent customer
3,764,9.0,Frequent customer
4,764,9.0,Frequent customer
5,777,11.0,Regular customer
6,825,20.0,Regular customer
7,910,6.0,Frequent customer
8,1052,10.0,Frequent customer
9,1052,10.0,Frequent customer


In [20]:
# exporting dataset as pickle format
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_.pkl'))